In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
#export
from nb_006b import *
from collections import Counter

# Wikitext 2

## Data

Download the dataset [here](https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip) and unzip it so it's in the folder wikitext.

In [ ]:
https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip

In [10]:
!aws s3 cp s3://research.metamind.io/wikitext/wikitext-2-v1.zip data

download: s3://research.metamind.io/wikitext/wikitext-2-v1.zip to data/wikitext-2-v1.zip


In [11]:
!unzip data/wikitext-2-v1.zip -d data

Archive:  data/wikitext-2-v1.zip
   creating: data/wikitext-2/
  inflating: data/wikitext-2/wiki.test.tokens  
  inflating: data/wikitext-2/wiki.valid.tokens  
  inflating: data/wikitext-2/wiki.train.tokens  


In [12]:
EOS = '<eos>'
PATH=Path('data/wikitext-2/')

Small helper function to read the tokens.

In [13]:
def read_file(filename):
    tokens = []
    with open(PATH/filename, encoding='utf8') as f:
        for line in f:
            tokens.append(line.split() + [EOS])
    return np.array(tokens)

In [14]:
train_tok = read_file('wiki.train.tokens')
valid_tok = read_file('wiki.valid.tokens')
test_tok = read_file('wiki.test.tokens')

In [26]:
train_tok[:5]

array([list(['<eos>']),
       list(['=', 'Valkyria', 'Chronicles', 'III', '=', '<eos>']),
       list(['<eos>']),
       list(['Senjō', 'no', 'Valkyria', '3', ':', '<unk>', 'Chronicles', '(', 'Japanese', ':', '戦場のヴァルキュリア3', ',', 'lit', '.', 'Valkyria', 'of', 'the', 'Battlefield', '3', ')', ',', 'commonly', 'referred', 'to', 'as', 'Valkyria', 'Chronicles', 'III', 'outside', 'Japan', ',', 'is', 'a', 'tactical', 'role', '@-@', 'playing', 'video', 'game', 'developed', 'by', 'Sega', 'and', 'Media.Vision', 'for', 'the', 'PlayStation', 'Portable', '.', 'Released', 'in', 'January', '2011', 'in', 'Japan', ',', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'Valkyria', 'series', '.', '<unk>', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', '@-@', 'time', 'gameplay', 'as', 'its', 'predecessors', ',', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the', '"', 'Nameless', '"', ',', 'a', 'penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 

In [27]:
len(train_tok), len(valid_tok), len(test_tok)

(36718, 3760, 4358)

In [28]:
' '.join(train_tok[4][:20])

'The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II'

In [32]:
cnt = Counter(word for sent in train_tok for word in sent)
cnt.most_common(10)

[('the', 113161),
 (',', 99913),
 ('.', 73388),
 ('of', 56889),
 ('<unk>', 54625),
 ('and', 50603),
 ('in', 39453),
 ('to', 39190),
 ('<eos>', 36718),
 ('a', 34237)]

Give an id to each token and add the pad token (just in case we need it).

In [40]:
itos = [o for o,c in cnt.most_common()]
itos.insert(0,'<pad>')

In [41]:
vocab_size = len(itos); vocab_size

33279

Creates the mapping from token to id then numericalizing our datasets.

In [35]:
stoi = collections.defaultdict(lambda : 5, {w:i for i,w in enumerate(itos)})

In [42]:
itos[5]

'<unk>'

In [43]:
train_ids = np.array([([stoi[w] for w in s]) for s in train_tok])
valid_ids = np.array([([stoi[w] for w in s]) for s in valid_tok])
test_ids = np.array([([stoi[w] for w in s]) for s in test_tok])

In [44]:
valid_ids.shape

(3760,)

In [47]:
valid_ids[:3]

array([list([9]), list([11, 10854, 33171, 11, 9]), list([9])],
      dtype=object)

In [53]:
#export
class LanguageModelLoader():
    "Creates a dataloader with bptt slightly changing."
    
    def __init__(self, nums:np.ndarray, bs:int=64, bptt:int=70, backwards:bool=False):
        self.bs,self.bptt,self.backwards = bs,bptt,backwards
        self.data = self.batchify(nums)
        self.first,self.i,self.iter = True,0,0
        self.n = len(self.data)

    def __iter__(self):
        self.i,self.iter = 0,0
        while self.i < self.n-1 and self.iter<len(self):
            if self.first and self.i == 0: self.first,seq_len = False,self.bptt + 25
            else:
                bptt = self.bptt if np.random.random() < 0.95 else self.bptt / 2.
                seq_len = max(5, int(np.random.normal(bptt, 5)))
            res = self.get_batch(self.i, seq_len)
            self.i += seq_len
            self.iter += 1
            yield res

    def __len__(self) -> int: return (self.n-1) // self.bptt

    def batchify(self, data:np.ndarray) -> LongTensor:
        "Splits the data in batches."
        nb = data.shape[0] // self.bs
        # breakpoint()
        # train
        # data.shape (2088628,)
        # np.array(data[:nb*self.bs]).shape (2088620,)
        # self.bs 20
        # np.array(data[:nb*self.bs]).reshape(self.bs, -1).shape (20, 104431)
        # np.array(data[:nb*self.bs]).reshape(self.bs, -1).T.shape (104431, 20)
        data = np.array(data[:nb*self.bs]).reshape(self.bs, -1).T
    
        if self.backwards: data=data[::-1]
        return LongTensor(data)

    def get_batch(self, i:int, seq_len:int) -> LongTensor:
        "Gets a batch of length `seq_len`"
        seq_len = min(seq_len, len(self.data) - 1 - i)
        return self.data[i:i+seq_len], self.data[i+1:i+1+seq_len].contiguous().view(-1)

In [54]:
bs,bptt = 20,10
train_dl = LanguageModelLoader(np.concatenate(train_ids), bs, bptt)
valid_dl = LanguageModelLoader(np.concatenate(valid_ids), bs, bptt)

In [72]:
self=train_dl

In [73]:
self.bptt

10

In [74]:
int(np.random.normal(self.bptt, 5))

9

In [75]:
seq_len = max(5, int(np.random.normal(bptt, 5)))
seq_len

15

In [ ]:
i=10

In [82]:
self.data[i:i+seq_len].shape

torch.Size([15, 20])

In [78]:
self.data[i+1:i+1+seq_len].shape

torch.Size([15, 20])

In [81]:
self.data[i+1:i+1+seq_len].contiguous().shape

torch.Size([15, 20])

In [80]:
self.data[i+1:i+1+seq_len].contiguous().view(-1).shape

torch.Size([300])

In [55]:
len(train_dl)

10443

In [39]:
data = DataBunch(train_dl, valid_dl)

## Model

### 1. Dropout

We want to use the AWD-LSTM from [Stephen Merity](https://arxiv.org/abs/1708.02182). First, we'll need all different kinds of dropouts. Dropout consists into replacing some coefficients by 0 with probability p. To ensure that the averga of the weights remains constant, we apply a correction to the weights that aren't nullified of a factor `1/(1-p)`.

In [16]:
#export
def dropout_mask(x:Tensor, sz:Collection[int], p:float):
    "Returns a dropout mask of the same type as x, size sz, with probability p to cancel an element."
    return x.new(*sz).bernoulli_(1-p).div_(1-p)

In [17]:
x = torch.randn(10,10)
dropout_mask(x, (10,10), 0.5)

tensor([[0., 0., 2., 2., 0., 0., 2., 2., 0., 2.],
        [0., 0., 2., 2., 2., 2., 2., 0., 2., 2.],
        [0., 0., 2., 0., 2., 0., 2., 0., 2., 2.],
        [2., 0., 2., 0., 0., 2., 2., 2., 0., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 2., 2., 0., 2., 2., 0., 0.],
        [2., 2., 2., 0., 2., 2., 0., 0., 0., 2.],
        [0., 2., 2., 0., 2., 0., 0., 0., 2., 0.],
        [2., 2., 0., 2., 2., 2., 2., 2., 2., 0.],
        [0., 0., 2., 0., 0., 0., 2., 2., 2., 0.]])

Once with have a dropout mask `m`, applying the dropout to `x` is simply done by `x = x * m`. We create our own dropout mask and don't rely on pytorch dropout because we want to nullify the coefficients on the batch dimension but not the token dimension (aka the same coefficients are replaced by zero for each word in the sentence). 

Inside a RNN, a tensor x will have three dimensions: seq_len, bs, vocab_size, so we create a dropout mask for the last two dimensions and broadcast it to the first dimension.

In [18]:
#export
class RNNDropout(nn.Module):
    "Dropout that is consistent on the seq_len dimension"
    
    def __init__(self, p:float=0.5):
        super().__init__()
        self.p=p

    def forward(self, x:Tensor) -> Tensor:
        if not self.training or self.p == 0.: return x
        m = dropout_mask(x.data, (1, x.size(1), x.size(2)), self.p)
        return x * m

In [19]:
dp_test = RNNDropout(0.5)
x = torch.randn(2,5,10)
x, dp_test(x)

(tensor([[[-0.7283,  0.7443, -1.8396, -2.7654,  0.3736,  0.6672,  1.2411,
            0.8020, -0.2302, -1.0877],
          [ 2.2003,  0.8253,  0.6388, -0.6059, -0.6410,  0.8697,  0.7311,
            0.1968,  0.3669, -0.1167],
          [-0.1711,  2.0904,  0.1090, -0.2663, -0.9678, -0.3236,  0.3334,
            0.7960, -0.1871, -2.5345],
          [-0.4101, -2.0172,  1.9497,  0.2155,  0.4823, -1.6743,  2.4850,
           -0.5896, -0.5311, -0.2382],
          [ 1.7063,  0.8261, -0.8135, -0.0204,  0.9086, -0.7822, -0.9808,
           -1.6546, -0.0143, -0.7513]],
 
         [[-0.6034,  1.2387,  0.7723,  0.2045, -0.5969,  1.0645, -0.0198,
            0.3751,  0.1990, -1.9619],
          [ 0.0461,  0.0713, -0.9382, -1.9040,  0.7329,  0.0634, -0.0106,
            1.5115,  1.2739,  0.3524],
          [-1.3722, -0.1292, -0.2983, -1.7732,  2.7751,  0.7476,  0.4863,
           -0.8636, -0.4760,  0.8205],
          [ 0.8279,  0.4220, -0.0188,  1.1219, -0.5122, -0.2561, -0.1091,
           -1.1657,

In [20]:
#export
import warnings

class WeightDropout(nn.Module):
    "A module that warps another layer in which some weights will be replaced by 0 during training."
    
    def __init__(self, module:Model, weight_p:float, layer_names:Collection[str]=['weight_hh_l0']):
        super().__init__()
        self.module,self.weight_p,self.layer_names = module,weight_p,layer_names
        for layer in self.layer_names:
            #Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
    
    def _setweights(self):
        "Applies dropout to the raw weights"
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = F.dropout(raw_w, p=self.weight_p, training=self.training)
            
    def forward(self, *args:ArgStar):
        self._setweights()
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            return self.module.forward(*args)
    
    def reset(self):
        if hasattr(self.module, 'reset'): self.module.reset()

In [21]:
module = nn.LSTM(20, 20)
dp_module = WeightDropout(module, 0.5)
opt = optim.SGD(dp_module.parameters(), 10)
dp_module.train()

WeightDropout(
  (module): LSTM(20, 20)
)

In [22]:
x = torch.randn(2,5,20)
x.requires_grad_(requires_grad=True)
h = (torch.zeros(1,5,20), torch.zeros(1,5,20))
for _ in range(5): x,h = dp_module(x,h)

In [23]:
getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')

(tensor([[-0.1631, -0.1101,  0.2537,  ..., -0.0874, -0.0000,  0.0000],
         [ 0.0985,  0.4257,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.0669,  ..., -0.2192, -0.2993, -0.0000],
         ...,
         [ 0.0644, -0.0000,  0.0000,  ..., -0.4386, -0.0000,  0.0000],
         [ 0.3466, -0.3525,  0.0000,  ...,  0.0000,  0.3990,  0.0000],
         [-0.4137, -0.0000,  0.0995,  ..., -0.4364, -0.1720,  0.1987]],
        grad_fn=<MulBackward0>), Parameter containing:
 tensor([[-0.0816, -0.0550,  0.1269,  ..., -0.0437, -0.2212,  0.0648],
         [ 0.0492,  0.2128,  0.2189,  ...,  0.1867, -0.1781,  0.1222],
         [ 0.2103,  0.0843, -0.0335,  ..., -0.1096, -0.1496, -0.0910],
         ...,
         [ 0.0322, -0.1078,  0.0856,  ..., -0.2193, -0.1736,  0.0777],
         [ 0.1733, -0.1762,  0.1076,  ...,  0.0995,  0.1995,  0.1258],
         [-0.2069, -0.1522,  0.0498,  ..., -0.2182, -0.0860,  0.0993]],
        requires_grad=True))

In [24]:
target = torch.randint(0,20,(10,)).long()
loss = F.nll_loss(x.view(-1,20), target)
loss.backward()
opt.step()

In [25]:
w, w_raw = getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')
w.grad, w_raw.grad

(None, tensor([[-0.0000, -0.0000,  0.0000,  ...,  0.0000,  0.0001, -0.0000],
         [ 0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
         [-0.0000,  0.0000, -0.0008,  ..., -0.0006,  0.0004, -0.0006],
         ...,
         [ 0.0000, -0.0001,  0.0000,  ..., -0.0006, -0.0001,  0.0000],
         [ 0.0001, -0.0000,  0.0002,  ...,  0.0002, -0.0001,  0.0005],
         [-0.0000,  0.0001, -0.0001,  ..., -0.0000,  0.0001, -0.0002]]))

In [26]:
getattr(dp_module.module, 'weight_hh_l0'),getattr(dp_module,'weight_hh_l0_raw')

(tensor([[-0.1631, -0.1101,  0.2537,  ..., -0.0874, -0.0000,  0.0000],
         [ 0.0985,  0.4257,  0.0000,  ...,  0.0000, -0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.0669,  ..., -0.2192, -0.2993, -0.0000],
         ...,
         [ 0.0644, -0.0000,  0.0000,  ..., -0.4386, -0.0000,  0.0000],
         [ 0.3466, -0.3525,  0.0000,  ...,  0.0000,  0.3990,  0.0000],
         [-0.4137, -0.0000,  0.0995,  ..., -0.4364, -0.1720,  0.1987]],
        grad_fn=<MulBackward0>), Parameter containing:
 tensor([[-0.0816, -0.0550,  0.1268,  ..., -0.0437, -0.2225,  0.0652],
         [ 0.0491,  0.2131,  0.2190,  ...,  0.1868, -0.1779,  0.1223],
         [ 0.2103,  0.0843, -0.0257,  ..., -0.1038, -0.1532, -0.0851],
         ...,
         [ 0.0321, -0.1066,  0.0855,  ..., -0.2137, -0.1728,  0.0777],
         [ 0.1727, -0.1762,  0.1053,  ...,  0.0977,  0.2009,  0.1212],
         [-0.2066, -0.1533,  0.0509,  ..., -0.2178, -0.0865,  0.1014]],
        requires_grad=True))

In [27]:
#export
class EmbeddingDropout(nn.Module):
    "Applies dropout in the embedding layer by zeroing out some elements of the embedding vector."
    
    def __init__(self, emb:Model, embed_p:float):
        super().__init__()
        self.emb,self.embed_p = emb,embed_p
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None: self.pad_idx = -1

    def forward(self, words:LongTensor, scale:Optional[float]=None) -> Tensor:
        if self.training and self.embed_p != 0:
            size = (self.emb.weight.size(0),1)
            mask = dropout_mask(self.emb.weight.data, size, self.embed_p)
            masked_embed = self.emb.weight * mask
        else: masked_embed = self.emb.weight
        if scale: masked_embed.mul_(scale)
        return F.embedding(words, masked_embed, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

In [28]:
enc = nn.Embedding(100,20, padding_idx=0)
enc_dp = EmbeddingDropout(enc, 0.5)

In [29]:
x = torch.randint(0,100,(25,)).long()

In [30]:
enc_dp(x)

tensor([[-2.7143,  0.0896,  1.5000,  2.1003,  0.0004, -3.6886,  2.0353, -2.4099,
         -0.9990,  1.4987,  1.3285, -2.3275, -0.9443,  1.8773,  0.7688, -1.6824,
          0.7480, -1.6076, -1.6210, -1.4068],
        [-0.0000,  0.0000,  0.0000, -0.0000,  0.0000, -0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000, -0.0000,  0.0000, -0.0000, -0.0000,
          0.0000, -0.0000, -0.0000, -0.0000],
        [-1.2547, -2.4881,  2.8908,  2.0062, -3.5332, -0.2465, -1.5952,  2.0468,
         -0.8801, -1.0021,  1.0353,  0.5487,  1.7424, -1.7696, -2.3166,  0.4942,
         -0.5003, -3.9225,  3.0615, -0.2013],
        [-0.0000, -0.0000, -0.0000,  0.0000,  0.0000, -0.0000,  0.0000,  0.0000,
          0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
         -0.0000,  0.0000,  0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000,
         -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
      

### 2. AWD-LSTM

In [31]:
#export
def repackage_var(h:Tensors) -> Tensors:
    "Detaches h from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(repackage_var(v) for v in h)

In [32]:
#export
class RNNCore(nn.Module):
    "AWD-LSTM/QRNN inspired by https://arxiv.org/abs/1708.02182"

    initrange=0.1

    def __init__(self, vocab_sz:int, emb_sz:int, n_hid:int, n_layers:int, pad_token:int, bidir:bool=False,
                 hidden_p:float=0.2, input_p:float=0.6, embed_p:float=0.1, weight_p:float=0.5, qrnn:bool=False):
        
        super().__init__()
        self.bs,self.qrnn,self.ndir = 1, qrnn,(2 if bidir else 1)
        self.emb_sz,self.n_hid,self.n_layers = emb_sz,n_hid,n_layers
        self.encoder = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.encoder_dp = EmbeddingDropout(self.encoder, embed_p)
        if self.qrnn:
            #Using QRNN requires cupy: https://github.com/cupy/cupy
            from qrnn import QRNNLayer
            self.rnns = [QRNNLayer(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                                   save_prev_x=True, zoneout=0, window=2 if l == 0 else 1, output_gate=True, 
                                   use_cuda=torch.cuda.is_available()) for l in range(n_layers)]
            if weight_p != 0.:
                for rnn in self.rnns:
                    rnn.linear = WeightDropout(rnn.linear, weight_p, layer_names=['weight'])
        else:
            self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz)//self.ndir,
                1, bidirectional=bidir) for l in range(n_layers)]
            if weight_p != 0.: self.rnns = [WeightDropout(rnn, weight_p) for rnn in self.rnns]
        self.rnns = torch.nn.ModuleList(self.rnns)
        self.encoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

    def forward(self, input:LongTensor) -> Tuple[Tensor,Tensor]:
        sl,bs = input.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()
        raw_output = self.input_dp(self.encoder_dp(input))
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output)
        self.hidden = repackage_var(new_hidden)
        return raw_outputs, outputs

    def one_hidden(self, l:int) -> Tensor:
        "Returns one hidden state"
        nh = (self.n_hid if l != self.n_layers - 1 else self.emb_sz)//self.ndir
        return self.weights.new(self.ndir, self.bs, nh).zero_()

    def reset(self):
        "Resets the hidden states"
        [r.reset() for r in self.rnns if hasattr(r, 'reset')]
        self.weights = next(self.parameters()).data
        if self.qrnn: self.hidden = [self.one_hidden(l) for l in range(self.n_layers)]
        else: self.hidden = [(self.one_hidden(l), self.one_hidden(l)) for l in range(self.n_layers)]

In [33]:
#export
class LinearDecoder(nn.Module):
    "To go on top of a RNN_Core module"
    
    initrange=0.1
    
    def __init__(self, n_out:int, n_hid:int, output_p:float, tie_encoder:Model=None, bias:bool=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.output_dp = RNNDropout(output_p)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input:Tuple[Tensor,Tensor]) -> Tuple[Tensor,Tensor,Tensor]:
        raw_outputs, outputs = input
        output = self.output_dp(outputs[-1])
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded, raw_outputs, outputs

In [34]:
#export
class SequentialRNN(nn.Sequential):
    "A sequential module that passes the reset call to its children."
    def reset(self):
        for c in self.children():
            if hasattr(c, 'reset'): c.reset()

In [39]:
#export
def get_language_model(vocab_sz:int, emb_sz:int, n_hid:int, n_layers:int, pad_token:int, tie_weights:bool=True, 
                       qrnn:bool=False, bias:bool=True, output_p:float=0.4, hidden_p:float=0.2, input_p:float=0.6, 
                       embed_p:float=0.1, weight_p:float=0.5) -> Model:
    "To create a full AWD-LSTM"
    rnn_enc = RNNCore(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token, qrnn=qrnn,
                 hidden_p=hidden_p, input_p=input_p, embed_p=embed_p, weight_p=weight_p)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(vocab_sz, emb_sz, output_p, tie_encoder=enc, bias=bias))

In [ ]:
tst_model = get_language_model(500, 20, 100, 2, 0, qrnn=True)
tst_model.cuda()

In [ ]:
x = torch.randint(0, 500, (10,5)).long()
z = tst_model(x.cuda())

In [ ]:
len(z)

### 3. Callback to train the model

In [ ]:
#export
@dataclass
class GradientClipping(Callback):
    "To do gradient clipping during training."
    learn:Learner
    clip:float

    def on_backward_end(self, **kwargs):
        if self.clip:  nn.utils.clip_grad_norm_(self.learn.model.parameters(), self.clip)

In [ ]:
#export
@dataclass
class RNNTrainer(Callback):
    "`Callback` that regroups lr adjustment to seq_len, AR and TAR"
    learn:Learner
    bptt:int
    alpha:float=0.
    beta:float=0.
    adjust:bool=True
    
    def on_loss_begin(self, last_output:Tuple[Tensor,Tensor,Tensor], **kwargs):
        #Save the extra outputs for later and only returns the true output.
        self.raw_out,self.out = last_output[1],last_output[2]
        return last_output[0]
    
    def on_backward_begin(self, last_loss:Rank0Tensor, last_input:Tensor, last_output:Tensor, **kwargs):
        #Adjusts the lr to the bptt selected
        if self.adjust: self.learn.opt.lr *= last_input.size(0) / self.bptt
        #AR and TAR
        if self.alpha != 0.:  last_loss += (self.alpha * self.out[-1].pow(2).mean()).sum()
        if self.beta != 0.:
            h = self.raw_out[-1]
            if len(h)>1: last_loss += (self.beta * (h[1:] - h[:-1]).pow(2).mean()).sum()
        return last_loss

In [ ]:
emb_sz, nh, nl = 400, 1150, 3
model = get_language_model(vocab_size, emb_sz, nh, nl, 0, input_p=0.6, output_p=0.4, weight_p=0.5, 
                           embed_p=0.1, hidden_p=0.2)
learn = Learner(data, model)

In [ ]:
learn.opt_fn = partial(optim.Adam, betas=(0.8,0.99))
learn.callbacks.append(RNNTrainer(learn, bptt, alpha=2, beta=1))
learn.callback_fns = [partial(GradientClipping, clip=0.12)]

In [ ]:
fit_one_cycle(learn, 1, 5e-3, (0.8,0.7), wd=1.2e-6)